# Informer model

In [1]:
#!pip install deepts_forecasting

### Import libraries

In [1]:
import numpy as np
import torch
from torch.utils.data import DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

from deepts_forecasting.utils.data import TimeSeriesDataSet
from deepts_forecasting.utils.data.encoders import TorchNormalizer
from deepts_forecasting.datasets import AirPassengersDataset
from deepts_forecasting.models.autoformer import Autoformer


D:\Anaconda3\envs\DeepTS_Forecasting\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Dataset

In [2]:
data = AirPassengersDataset().load()
data['year'] = data['Month'].dt.year
data['month'] = data['Month'].dt.month
data['group'] = '0'
data['time_idx'] = np.arange(len(data))
data['Passengers'] = data['Passengers'].astype(float)
data['month'] = data['month'].astype('str')
data.head()

,Month,Passengers,year,month,group,time_idx
0,1949-01-01,112.0,1949,1,0,0
1,1949-02-01,118.0,1949,2,0,1
2,1949-03-01,132.0,1949,3,0,2
3,1949-04-01,129.0,1949,4,0,3
4,1949-05-01,121.0,1949,5,0,4


### Split train/test sets

In [3]:
max_encoder_length = 18
max_prediction_length = 12

training_cutoff = data["time_idx"].max() - max_encoder_length - max_prediction_length

training = TimeSeriesDataSet(
    data[lambda x: x.time_idx <= training_cutoff],
    max_encoder_length= max_encoder_length,
    min_encoder_length=max_encoder_length,
    max_prediction_length=max_prediction_length,
    min_prediction_length=max_prediction_length,
    time_idx="time_idx",
    target="Passengers",
    group_ids=["group"],
    static_categoricals=[],
    static_reals=[],
    time_varying_known_categoricals=['month'],
    time_varying_known_reals=[],
    time_varying_unknown_reals=["Passengers"],
    time_varying_unknown_categoricals=[],
    target_normalizer=TorchNormalizer(method="standard",
                                      transformation=None),
    )

training.get_parameters()
validation = TimeSeriesDataSet.from_dataset(training,
                                            data[lambda x: x.time_idx > training_cutoff])

batch_size = 16
train_dataloader = DataLoader(training, batch_size=batch_size, shuffle=False, drop_last=False)
val_dataloader = DataLoader(validation, batch_size=batch_size, shuffle=False, drop_last=False)


### Define model

In [4]:
pl.seed_everything(1234)
# create PyTorch Lighning Trainer with early stopping
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4,
                                    patience=60, verbose=False, mode="min")
lr_logger = LearningRateMonitor()

trainer = pl.Trainer(
    max_epochs=300,
    gpus=0,  # run on CPU, if on multiple GPUs, use accelerator="ddp"
    gradient_clip_val=0.1,
    limit_train_batches=30,  # 30 batches per epoch
    callbacks=[lr_logger, early_stop_callback],
    logger=TensorBoardLogger("lightning_logs")
)

model = Autoformer.from_dataset(training,
                                    d_model=28,
                                    d_ff=56,
                                    n_heads=2,
                                    num_layers=2,
                                    moving_avg=12,
                                      )
model.summarize

Global seed set to 1234
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


<bound method LightningModule.summarize of Autoformer(
  (loss): L1Loss()
  (logging_metrics): ModuleList()
  (decomp): SeriesDecompose(
    (moving_avg): MovingAvg(
      (avg): AvgPool1d(kernel_size=(12,), stride=(1,), padding=(0,))
    )
  )
  (embeddings): ModuleDict(
    (month): Embedding(12, 6)
  )
  (encoder_input_linear): TokenEmbedding(
    (tokenConv): Conv1d(7, 28, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
  )
  (decoder_input_linear): TokenEmbedding(
    (tokenConv): Conv1d(6, 28, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
  )
  (out_linear): Linear(in_features=28, out_features=1, bias=True)
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0): EncoderLayer(
        (attention): AutoCorrelationLayer(
          (inner_correlation): AutoCorrelation(
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (query_projection): Linear(in_features=28, out_features=28, bias=True

In [5]:
model.hparams

"activation":                        relu
"categorical_groups":                {}
"d_ff":                              56
"d_model":                           28
"dropout":                           0.1
"embedding_labels":                  {'month': array(['1', '10', '11', '12', '2', '3', '4', '5', '6', '7', '8', '9'],
      dtype=object)}
"embedding_paddings":                []
"embedding_sizes":                   {'month': [12, 6]}
"factor":                            5
"learning_rate":                     0.001
"log_interval":                      -1
"log_val_interval":                  None
"logging_metrics":                   ModuleList()
"loss":                              L1Loss()
"max_encoder_length":                18
"max_prediction_length":             12
"monotone_constaints":               {}
"moving_avg":                        12
"n_heads":                           2
"num_layers":                        2
"output_size":                       1
"output_transformer":    

### Train model with early stopping

In [6]:
trainer.fit(
    model, train_dataloader=train_dataloader, val_dataloaders=val_dataloader,
)

# (given that we use early stopping, this is not necessarily the last epoch)
best_model_path = trainer.checkpoint_callback.best_model_path
best_model = Autoformer.load_from_checkpoint(best_model_path)

# calcualte mean absolute error on validation set
actuals = torch.cat([model.transform_output(prediction=y, target_scale=x['target_scale'])
                     for x, y in iter(val_dataloader)])
predictions, x_index = best_model.predict(val_dataloader)
mae = (actuals - predictions).abs().mean()
# print('predictions shape is:', predictions.shape)
# print('actuals shape is:', actuals.shape)
print(torch.cat([actuals, predictions]))
print('MAE is:', mae)


D:\Anaconda3\envs\DeepTS_Forecasting\lib\site-packages\pytorch_lightning\trainer\trainer.py:735: LightningDeprecationWarning: `trainer.fit(train_dataloader)` is deprecated in v1.4 and will be removed in v1.6. Use `trainer.fit(train_dataloaders)` instead. HINT: added 's'
  rank_zero_deprecation(

  | Name                 | Type            | Params
---------------------------------------------------------
0 | loss                 | L1Loss          | 0     
1 | logging_metrics      | ModuleList      | 0     
2 | decomp               | SeriesDecompose | 0     
3 | embeddings           | ModuleDict      | 72    
4 | encoder_input_linear | TokenEmbedding  | 588   
5 | decoder_input_linear | TokenEmbedding  | 504   
6 | out_linear           | Linear          | 29    
7 | encoder              | Encoder         | 12.8 K
8 | decoder              | Decoder         | 24.0 K
---------------------------------------------------------
38.0 K    Trainable params
0         Non-trainable params
38.0 K   

D:\Anaconda3\envs\DeepTS_Forecasting\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 1234
D:\Anaconda3\envs\DeepTS_Forecasting\lib\site-packages\pytorch_lightning\trainer\data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
D:\Anaconda3\envs\DeepTS_Forecasting\lib\site-packages\pytorch_lightning\trainer\data_loading.py:432: UserWarning: The number of training samples (6) is smaller than the logging interval Trainer(log_every_n_s

Epoch 0:  86%|████████▌ | 6/7 [00:00<00:00, 11.97it/s, loss=1.23, v_num=24] 
Validating: 0it [00:00, ?it/s]
Epoch 1:  86%|████████▌ | 6/7 [00:00<00:00,  9.66it/s, loss=1.19, v_num=24, val_loss=2.990, train_loss=1.120]
Validating: 0it [00:00, ?it/s]
Epoch 2:  86%|████████▌ | 6/7 [00:00<00:00,  7.78it/s, loss=1.06, v_num=24, val_loss=5.780, train_loss=0.958]
Validating: 0it [00:00, ?it/s]
Epoch 3:  86%|████████▌ | 6/7 [00:00<00:00,  6.96it/s, loss=1.02, v_num=24, val_loss=0.821, train_loss=0.763] 
Validating: 0it [00:00, ?it/s]
Epoch 4:  86%|████████▌ | 6/7 [00:00<00:00,  8.16it/s, loss=0.876, v_num=24, val_loss=2.830, train_loss=0.893]
Validating: 0it [00:00, ?it/s]
Epoch 5:  86%|████████▌ | 6/7 [00:00<00:00,  8.89it/s, loss=0.816, v_num=24, val_loss=4.130, train_loss=0.631]
Validating: 0it [00:00, ?it/s]
Epoch 6:  86%|████████▌ | 6/7 [00:00<00:00,  9.79it/s, loss=0.715, v_num=24, val_loss=1.370, train_loss=0.674]
Validating: 0it [00:00, ?it/s]
Epoch 7:  86%|████████▌ | 6/7 [00:00<00:00